# Exercise 2

For this exercise the LSH algorithm was developed to identify similar news articles. This algorithm was implemented using Spark, more specifically the PySpark library with the Dataframe API.

## Imports

PySpark is the only non-standard library required.

In [72]:
import os.path
import random
import math
import pyspark.sql.functions as F
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StringType, ArrayType, IntegerType
from itertools import combinations
from typing import Iterable, Any, List, Callable

## Parameters

The values for parameters $b$ and $r$ chosen, according to the requirements (2.1), were:
- $b = 13$
- $r = 11$

The values were hand-picked by visually analyzing the plot for the probability of two documents sharing a bucket depending on their similarity, as $b$ and $r$ changed.

In [73]:
N = 100

r = 11
b = 13

point_below = (0.85, 0.9)
point_above = (0.6, 0.05)

prob = lambda s, r, b: 1 - (1 - s**r)**b

try:
    import matplotlib.pyplot as plt

    ss = [i/N for i in range(N)]

    plt.plot(*point_below, color='g', marker='o')
    plt.plot(*point_above, color='r', marker='o')
    plt.plot(ss, [prob(s, r, b) for s in ss])

    plt.title(f'Probability of two documents sharing a bucket w.r.t. their similarity $s$\n($r={r}$, $b={b}$)')
    plt.legend(['at least', 'less than', 'probability'])
    plt.xlabel('$s$')
    plt.ylabel('probability')

    plt.show()

except ImportError:
    print('Could not plot, since the \'matplotlib\' module is not present.')

assert prob(point_below[0], r, b) >= point_below[1], 'Pairs with a similarity of 85%% should have at least 90%% probability of sharing a bucket!'
assert prob(point_above[0], r, b) <  point_above[1], 'Pairs with a similarity of 60%% should have less than 5%% probability of sharing a bucket!'

Could not plot, since the 'matplotlib' module is not present.


Here we have defined all the algorithm's paramters.

In [74]:
# Shingle size
k = 9

# Number of bands
b = 13

# Number of rows per band
r = 11

# Min-hash: number of hash functions
num_functions = b*r

# Seed for the random number generator
seed = 1

# Similarity threshold
similarity_threshold = 0.85

# Number of explicit partitions
num_partitions = 8

The seed for the random number generator is set, which will be used when generating the min-hash hash function family and obtaining a sample of the dataset for false positive/negative analysis.

In [75]:
random.seed(seed)

## Spark Initialization

Spark is initialized, with as many worker threads as logical cores on the machine.
We did not use a fixed value since the machines used for development had a different number of CPU cores.

In [76]:
spark = SparkSession.builder \
    .appName('LSH') \
    .config('spark.master', 'local[*]') \
    .getOrCreate()

## Prepare the Data

The dataset is about news in Twitter, where each row identifies a tweet ID, URL and text.

The data's format is JSON, and is loaded to a dataframe.
The data is partitioned using a fixed number of partitions, and it will be repartitioned again in the future.
This helps alleviate the impact that `filter` operations have on the partitions, which were heavily hampering the algorithm's performance.

In [77]:
df = spark.read \
    .json('./data/covid_news_small.json.bz2') \
    .repartition(num_partitions)

The dataframe `df` will have three columns: `text`, `tweet_id` and `url`.

## Pipeline

### Generate shingles

The first step for the algorithm is to generate the shingles for each document/tweet.
We acomplish this by removing all the tweets which won't have at least one shingle of size `k` using a filter.
The data is partitioned after filtering to avoid data skew.

Then we use a UDF to create all the shingles of each `text`.
Within the UDF, each shingle which will also be hashed to a 32 bit integer, so that it can be stored in a Spark `IntegerType`.

In [78]:
@F.udf(returnType=ArrayType(IntegerType(), False))
def generate_shingles(text: str):
    shingles = (text[idx:idx+k] for idx in range(len(text) - k + 1))
    # Get last 32 bits in order to have 4-byte integers (Python allows arbitrarily large integers)
    to_integer = lambda s: hash(s) & ((1 << 32) - 1)
    return list(set(map(to_integer, shingles)))

In [79]:
df_shingles = df \
    .drop('url') \
    .filter(F.length('text') >= k) \
    .repartition(num_partitions) \
    .withColumn('shingles', generate_shingles('text')) \
    .drop('text')

With this, the dataframe `df_shingles` will be composed of two columns: `tweet_id` and `shingles`, the latter being an array of the hashed shingles for this tweet.

### Min-hash

The next step is to generate the min-hash signatures.

First we need to generate the hash functions. We will use the following function to generate `num_functions` hash functions from a universal hash family of the form
$$
((a \times x + b) \operatorname{mod} p) \operatorname{mod} N
$$.
Our `N` is the number of possible shingles (in this case our hashed shingles are 32-bit integers, so `N` is $2^{32}$), and `p` is a prime number larger than `N`.

We randomly generate unique pairs of `a` and `b`, and the hash functions will be represented by all the four parameters (even though `p` and `N` end up being the same).

In [80]:
# Assumes the values to hash are 4-byte integers
def generate_universal_hash_family(K: int) -> List[Callable[[int], int]]:
    N = 1 << 32
    p = 2305843009213693951

    parameters = set()
    while (len(parameters) < K):
        parameters |= {(random.randint(1, N), random.randint(0, N)) for _ in range(K - len(parameters))}
    
    return [(a, b, p, N) for a, b in parameters]

We explicitly broadcast the generated hash functions to all nodes, so that the hash parameters can be easily accessed.

In [81]:
hash_family = generate_universal_hash_family(num_functions)
broadcasted_hash_family = spark.sparkContext.broadcast(hash_family)

Then we just need to use the generated hash functions to calculate the min-hash signatures for each tweet.
An UDF was used, especially since Python allows arbitrarily large integers, and the numbers involved in calculating the hash values are very large.

In [82]:
@F.udf(returnType=ArrayType(IntegerType(), False))
def calculate_min_hash(shingles: List[int]):
    return [min(((a * shingle + b) % p) % N for shingle in shingles) for (a, b, p, N) in broadcasted_hash_family.value]

In [83]:
def create_df_minhash(df_shingles: DataFrame) -> DataFrame:
    return df_shingles.withColumn('min_hash', calculate_min_hash('shingles')).drop('shingles')

df_minhash = create_df_minhash(df_shingles)

With this, the dataframe `df_minhash` will be composed of two columns: `tweet_id` and `min_hash`, where `tweet_id` is the ID of the document/tweet and `min_hash` is a list of integers, each one being the result of applying one of the hash functions to the shingles of the document/tweet calculated using the `calculate_min_hash` UDF.

The min-hash results are saved in disk in Parquet format (Spark's default format) for later use.

In [85]:
fname_minhash = f'minhash_{r}_{b}'
if not os.path.exists(fname_minhash):
    df_minhash.write.mode('overwrite').parquet(path=fname_minhash, compression='gzip')

df_minhash = spark.read.parquet(fname_minhash)

### LSH

The last step is to apply the LSH algorithm to the min-hashes and obtain the candidate pairs.

First we need to divide the min-hash signatures into `b` bands, each of size `r`. We developed the following UDF to do so.

In [86]:
@F.udf(returnType=ArrayType(ArrayType(IntegerType(), False), False))
def generate_even_slices(minhashes: List[int]):
    return [minhashes[i:i+r] for i in range(0, num_functions, r)]

After dividing the min-hash signatures in even slices with the previous UDF, we need to hash all the min-hash values of each band to obtain the band-specific bucket identifiers.
For this we use the hash function of the Spark library, creating a column named `bands` which will have an array of pairs, with 2 items:
- bucket identifier (`band_hash`), which is the hash value applied to a given band;
- the `band` number/identifier to which the bucket is associated to (integer in $[0, b)$).

This array is pairs is exploded into rows, one for each pair, which will allow grouping operations in the future.

At the end of the function we separate the `band_hash` and `band` columns into two different columns, one for each.

In [87]:
def create_df_bands(df_minhash: DataFrame) -> DataFrame:
    return df_minhash \
        .withColumn('min_hash_slices', generate_even_slices('min_hash')) \
        .withColumn('bands', F.array(*(
            F.struct(
                F.hash(F.col('min_hash_slices')[band]).alias('band_hash'),
                F.lit(band).alias('band')
            )
            for band in range(b))
        )) \
        .withColumn('bands', F.explode('bands')) \
        .select('tweet_id', F.col('bands').band.alias('band'), F.col('bands').band_hash.alias('band_hash'))

df_bands = create_df_bands(df_minhash)

This leaves us with the dataframe `df_bands`, which will be composed of three columns: `tweet_id`, `band` and `band_hash`, the latter being the bucket identifier.

For the next step, we generate the candidate pairs, and so we reuse the pair-generating UDF from exercise 1.

In [88]:
@F.udf(returnType=ArrayType(ArrayType(StringType(), False), False))
def combine_pairs(elems: Iterable[Any]):
    return list(combinations(elems, 2))

Having the buckets for each document/tweet in a band, we can now generate the pairs of documents/tweets that are candidates for being similar.

For this, we begin by grouping the documents/tweets by `band` and `band_hash` (that is, group the documents that are in the same bucket for a given band).
Then, the tweets are aggregated into an array column, called `candidates`.

Then we sort the `candidates` arrays to facilitate the removal of duplicate pairs, and filter the rows that have only one tweet ot less.

By doing selecting the distinct `candidates` we can remove plenty of buckets that report the same tweets, avoiding possible memory issues when generating pairs (subject to combinatorial explosion).
Finally, after generating the pairs within each bucket, we explode the `candidates` column to get the pairs of tweets and separate them into two columns, named `candidate_pair_first` and `candidate_pair_second`.
We additionally remove the duplicates generated by the combinations.

In [89]:
def create_df_candidate_pairs(df_bands: DataFrame) -> DataFrame:
    return df_bands \
        .groupby('band', 'band_hash') \
        .agg(F.collect_list('tweet_id')) \
        .withColumnRenamed('collect_list(tweet_id)', 'candidates') \
        .withColumn('candidates', F.array_sort('candidates')) \
        .filter(F.size('candidates') > 1) \
        .repartition(num_partitions) \
        .select('candidates') \
        .distinct() \
        .select(F.explode(combine_pairs('candidates')).alias('candidate_pair')) \
        .select(F.col('candidate_pair')[0].alias('candidate_pair_first'), F.col('candidate_pair')[1].alias('candidate_pair_second')) \
        .distinct() 

df_candidate_pairs = create_df_candidate_pairs(df_bands)

Lastly, before saving the results on disk, we remove the false positives, that is, the candidate pairs that have a Jaccard similarity, considering the shingles, above 85%.
The results saved on disk will be useful to quickly query the `get_similar_articles` functions developed further for exercise 2.2.

To verify if a given pair is a false positive, we calculate the Jaccard similarity documents' shingles in each pair using the `df_shingles` dataframe.
This dataframe will be persisted in Spark using the default storage level (`MEMORY_AND_DISK`).
It's not saved directly into disk since it has a size proportional to the source dataset.

In [91]:
def create_df_candidate_pairs_fpless(df_candidate_pairs: DataFrame, df_shingles: DataFrame, similarity_threshold: float) -> DataFrame:
    return df_candidate_pairs \
        .join(df_shingles, df_shingles['tweet_id'] == F.col('candidate_pair_first')) \
        .withColumnRenamed('shingles', 'shingles_first') \
        .drop('tweet_id') \
        .join(df_shingles, df_shingles['tweet_id'] == F.col('candidate_pair_second')) \
        .withColumnRenamed('shingles', 'shingles_second') \
        .drop('tweet_id') \
        .withColumn('similarity', F.size(F.array_intersect('shingles_first', 'shingles_second')) / F.size(F.array_union('shingles_first', 'shingles_second'))) \
        .drop('shingles_first', 'shingles_second') \
        .filter(F.col('similarity') >= similarity_threshold)

df_shingles.cache()
df_candidate_pairs_fpless = create_df_candidate_pairs_fpless(df_candidate_pairs, df_shingles, similarity_threshold)

This leaves us with the dataframe `df_candidate_pairs_fpless`, which will be composed of five columns: `candidate_pair_first`, `candidate_pair_second` and `similarity`.
The results are saved into disk, considering the integer percentage of the similarity threshold that was initially defined.

In [92]:
fname_candidate_pairs = f'candidate_pairs_{r}_{b}_{int(similarity_threshold * 100)}'
if not os.path.exists(fname_candidate_pairs):
    df_candidate_pairs_fpless.write.mode('overwrite').parquet(path=fname_candidate_pairs, compression='gzip')

df_candidate_pairs_fpless = spark.read.parquet(fname_candidate_pairs)

## Procedure to get similar articles

For exercise 2.2 we developed a function to get articles similar to a given article, identified by the tweet ID.

In this function we filter all the pairs which have the given document/tweet_id and create a Python `list` with all the similar articles.

Starting with the dataframe of candidate pairs without false positives, which is saved in disk, we transform both columns relating to the candidate pairs such that tweets equal to the queried-for tweet are turned to `null` (if `.otherwise()` is not specified after `.when()`, then all values that don't meet the condition in the `.when()` function are converted to `null`).

Afterwards, all rows that don't have at least a `null` value in one of the columns (i.e. all candidate pairs that don't have the queried-for tweet) are filtered-out.
Finally, both columns relating to the pair elements are coalesced, giving the first non-`null` value, which is the tweet that is not the queried-for tweet.

In [93]:
def get_similar_articles(tweet_id: str) -> List[str]:
    rows = df_candidate_pairs_fpless \
        .withColumn('candidate_pair_first', F.when(F.col('candidate_pair_first') != tweet_id, F.col('candidate_pair_first'))) \
        .withColumn('candidate_pair_second', F.when(F.col('candidate_pair_second') != tweet_id, F.col('candidate_pair_second'))) \
        .filter(F.col('candidate_pair_first').isNull() | F.col('candidate_pair_second').isNull()) \
        .select(F.coalesce('candidate_pair_first', 'candidate_pair_second').alias('similar_article')) \
        .collect()

    return [row.similar_article for row in rows]

## Analysis of false positives/negatives

In [ ]:
# Sample of the dataset to use
sample_fraction = 0.1

Here we load sample of the shingles data to do the analysis of false positives and negatives.

Then we generate the candidate pairs like before.
All necessary code was enclosed in functions, to facilitate this step.

In [94]:
df_shingles_sample = df_shingles.sample(fraction=sample_fraction, seed=seed, withReplacement=False)

df_minhash_sample = create_df_minhash(df_shingles_sample)

df_candidate_pairs_sample = create_df_candidate_pairs(create_df_bands(df_minhash_sample))

df_candidate_pairs_fpless_sample = create_df_candidate_pairs_fpless(df_candidate_pairs_sample, df_minhash_sample, similarity_threshold)

### False positive percentage (false discovery rate)

Since we have the dataframe of candidate pairs and the dataframe of candidate pairs without false positives, we can get the number of false positives by subtracting the number of rows of the dataframes.

In [95]:
candidate_pairs_n = df_candidate_pairs_sample.count()
candidate_pairs_fpless_n = df_candidate_pairs_fpless_sample.count()
print(f'Percentage of false positives: {(candidate_pairs_n - candidate_pairs_fpless_n) / candidate_pairs_n:%}')

Percentage of false positives: 19.322567%


### False negative percentage (false omission rate).

In order to get the false negative percentage, pairs between elements of the `df_shingles` dataframe are created (which are all possible pairs of documents) using a `crossJoin` with itself.

Only sorted pairs are used, so that effectively equal pairs (such as `(A, B)` and `(B, A)`) and pairs of one element (such as `(A, A)`) are not used.
They are sorted in ascending order, for proper comparison with the candidate pairs.

Afterwards, we join with the dataframe of candidate pairs, with the goal of removing all rows that match with said dataframe (to get all pairs that weren't candidates).
To do so, we perform a `left` join on the document pairs of both dataframes.

Since it's a `left` join, rows that don't match with the `df_candidate_pairs_sample` dataframe will have `null` values for the columns `candidate_pair_first` and `candidate_pair_second`.
Therefore, to get all rows that don't match with `df_candidate_pairs_sample`, we keep the rows with `null` values on the previously mentioned columns (it's enough to evaluate only one of them, since if one of them is `null` then the other is necessarily `null` as well).

With this, the Jaccard similarity between the remaining pairs is calculated, using the shingles.
Finally, the pairs are filtered in order to get those that surpassed the similarity threshold, and are thus false negatives.

In [96]:
false_negatives = df_shingles_sample \
    .crossJoin(df_shingles_sample.select(F.col('tweet_id').alias('tweet_id_other'), F.col('shingles').alias('shingles_other'))) \
    .filter(F.col('tweet_id') < F.col('tweet_id_other')) \
    .join(df_candidate_pairs_sample, (F.col('tweet_id') == F.col('candidate_pair_first')) & (F.col('tweet_id_other') == F.col('candidate_pair_second')), 'left') \
    .filter(F.col('candidate_pair_first').isNull()) \
    .drop('candidate_pair_first', 'candidate_pair_second') \
    .withColumn('similarity', F.size(F.array_intersect('shingles_first', 'shingles_second')) / F.size(F.array_union('shingles_first', 'shingles_second'))) \
    .filter(F.col('similarity') >= similarity_threshold) \
    .count()

To calculate the final percentage of false negatives we used the count previously calculated and divide it by the total number of negatives detected, which is obtained by subtracting the number of candidate pairs to the number of possible pairs of documents.

In [97]:
print(f'Percentage of false negatives: {false_negatives / (math.comb(df_minhash_sample.count(), 2) - candidate_pairs_n):%}')

Percentage of false negatives: 0.000000%
